In [1]:
from mpi4py import MPI
import plannerMethods
import time
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from math import ceil
from typing import Tuple, List, Dict
import numpy as np
import pandas as pd
import pickle
import csv
import os
import sys
import yaml
import random
import abc
from agents.stockItems.Deposit import Deposit
from agents.stockItems.Reserve import Reserve
from agents.stockItems.Loan import Loan
from agents.stockItems.ConsumptionGood import ConsumptionGood

from agents.BasicAgent import BasicAgent
from agents.CentralBank import CentralBank
from agents.CommercialBank import CommercialBank
from agents.Household import Household
from agents.AgentReporter import AgentReporter
from agents.CentralPlanner import CentralPlanner
from agents.Firm import Firm

from markets.CreditMarket import CreditMarket
from markets.ConsumptionMarket import ComsumptionMarket
from markets.LaborMarket import LaborMarket
from markets.MarketReporter import MarketReporter



In [2]:
comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size()


startTime=-1
def T():
    global startTime
    if startTime < 0:
        startTime=time.time()
    return time.time() - startTime
T() #launches the timer

#cpuTimer Tc()
startCpuTime=-1
def Tc():
    global startCpuTime
    if startCpuTime < 0:
        startCpuTime=time.process_time()
    return time.process_time() - startCpuTime
Tc() #launches the cpu timer



model_params = parameters.init_params("model1_general/config/model/model1.yaml", "")
agent_params = parameters.init_params("model1_general/config/agent/ini_agent.yaml", "")
params = {**model_params, **agent_params}

root_path = 'model1_general/'
log_path = root_path + params["log_file_root"]

repast4py.random.init(rng_seed=params['myRandom.seed'][rank]) #each rank has a seed
rng = repast4py.random.default_rng 

if os.path.isdir(log_path+"."+str(rank)):
    os.system("rm -R "+log_path+"."+str(rank))
os.makedirs(log_path+"."+str(rank))

#copy in the output folder the starting set of parameters
os.system(f"cp {root_path}model1.yaml "+log_path+"."+str(rank)+"/")
os.system(f"cp {root_path}firm-features.csv "+log_path+"."+str(rank)+"/")
os.system(f"cp {root_path}plannerMethods.py "+log_path+"."+str(rank)+"/")

if rank==0:
    i=0
    while os.path.isdir(log_path+"."+str(rankNum+i)):
        os.system("rm -R "+log_path+"."+str(rankNum+i))
        i+=1
    

    
#moves to the right folder (that you must create and initialize with a firm-features.csv file)
if not os.path.isdir(log_path+"."+str(rank)):
    print("There is no "+log_path+"."+str(rank) + " starting folder!")  
    sys.exit(0)
else: os.chdir(log_path+"."+str(rank))


FileNotFoundError: [Errno 2] No such file or directory: 'model1_general/config/model/model1.yaml'

In [ ]:
class GlobalContainer:
    def __init__(self, params):
        
        '''
        {agent_type: 
            rankid: real agent uid for this type}

        '''

        self.real_agents = dict()
        self.all_agents = dict()
        
        # create unique agents
        for agent_type, agent_config in params['ini_agents_unique'].items():
            self.real_agents[agent_type] = {i: [] for i in range(rankNum)}
            self.all_agents[agent_type] = {i: [] for i in range(rankNum)}
            
            if agent_config['isGlobal']:
                i_rank = agent_config['specific_rank'] if agent_config['specific_rank'] else 0
                
                agent_tuple = ((0, agent_type, i_rank), 
                                           (agent_config['paramGroup'][0], agent_config['resourceGroup'], agent_config['isGlobal']))
                self.real_agents[agent_type][i_rank].append(agent_tuple)
                # Add to all ranks if global
                for j_rank in range(rankNum):
                    self.all_agents[agent_type][j_rank].append(agent_tuple)
            else:
                if len(agent_config['paramGroup']) == 1:
                    for j_rank in range(rankNum):
                        agent_tuple = ((0, agent_type, j_rank), 
                                           (agent_config['paramGroup'][0], agent_config['resourceGroup'], agent_config['isGlobal']))
                        self.real_agents[agent_type][j_rank].append(agent_tuple)
                        self.all_agents[agent_type][j_rank].append(agent_tuple)
                else:
                    if len(agent_config['paramGroup']) == rankNum:
                        for j_rank in range(rankNum):
                            agent_tuple = ((0, agent_type, j_rank), 
                                               (agent_config['paramGroup'][j_rank], agent_config['resourceGroup'], agent_config['isGlobal']))
                            self.real_agents[agent_type][j_rank].append(agent_tuple)
                            self.all_agents[agent_type][j_rank].append(agent_tuple)
                    else:
                        raise ValueError('the length of paramGroup list for unique agent should define equal to rank number or 1')
            
            for rk in range(rankNum):
                assert len(self.all_agents[agent_type][rk]) == 1
        
        # create not unique agents
        if params['agent_distribute_mode'] == 'mean':
            for agent_type, agent_config in params['ini_agents_uniform'].items():
                self.real_agents[agent_type] = {i: [] for i in range(rankNum)}
                self.all_agents[agent_type] = {i: [] for i in range(rankNum)}
                
                total_amount = agent_config['total_amount']
                
                weights = np.ones(rankNum)/rankNum
                if agent_config['rank_weight'] is not None:
                    if len(agent_config['rank_weight']) == rankNum:
                        weights = np.array(agent_config['rank_weight'])
                        weights = weights / weights.sum()
                    else:
                        raise ValueError('the length of rank_weight list should define equal to rank number')

                base_amount_per_rank = np.floor(total_amount * weights)
                # remainder = total_amount - sum(base_amount_per_rank)
                
                for i_rank in range(rankNum):
                    # if i_rank < remainder:
                    #     agents_this_rank = base_amount_per_rank[i_rank] + 1 
                    # else:
                    #     agents_this_rank = base_amount_per_rank[i_rank]
                    agents_this_rank = base_amount_per_rank[i_rank]
                    
                    
                    groups_amount = self.calculate_group_amounts(agent_config['local_distribute'], agents_this_rank, agent_type)
                    
                    id_count = 0
                    for group_name, group_config in agent_config['local_distribute'].items():
                        amount = groups_amount[group_name]
                        for _ in range(amount):
                            agent_tuple = ((id_count, agent_type, i_rank), 
                                           (group_config['paramGroup'], group_config['resourceGroup'], group_config['isGlobal']))
                            self.real_agents[agent_type][i_rank].append(agent_tuple)
                            if group_config['isGlobal']:
                                # Add to all ranks if global
                                for j_rank in range(rankNum):
                                    self.all_agents[agent_type][j_rank].append(agent_tuple)
                            else:
                                # Add only to this rank if not global
                                self.all_agents[agent_type][i_rank].append(agent_tuple)
                        
                            id_count += 1
        else:
            for i_rank, rank_config in params['ini_agents_specific'].items():
                
                for agent_type, agent_config in rank_config.items():
                    if agent_type not in self.real_agents:
                        self.real_agents[agent_type] = {i: [] for i in range(rankNum)}
                        self.all_agents[agent_type] = {i: [] for i in range(rankNum)}
                
                    agents_this_rank = agent_config['total_amount']
                
                    groups_amount = self.calculate_group_amounts(agent_config['local_distribute'], agents_this_rank, agent_type)
                    
                    id_count = 0
                    for group_name, group_config in agent_config['local_distribute'].items():
                        amount = groups_amount[group_name]
                        for _ in range(amount):
                            agent_tuple = ((id_count, agent_type, i_rank), 
                                           (group_config['paramGroup'], group_config['resourceGroup'], group_config['isGlobal']))
                            self.real_agents[agent_type][i_rank].append(agent_tuple)
                            if group_config['isGlobal']:
                                # Add to all ranks if global
                                for j_rank in range(rankNum):
                                    self.all_agents[agent_type][j_rank].append(agent_tuple)
                            else:
                                # Add only to this rank if not global
                                self.all_agents[agent_type][i_rank].append(agent_tuple)
                        
                            id_count += 1

    def calculate_group_amounts(self, groups, total_amount, agent_type):
        group_amounts = {}
        total_share = sum(group['share'] for group in groups.values())

        if total_share != 1:
            print(f'the total share of agent type {agent_type} not equal to 1, we will rebalance the share to 1')

        for group_name, group_config in groups.items():
            share = group_config['share']/total_share
            amount = int(total_amount * share)
            group_amounts[group_name] = amount

        return group_amounts
    
    def getAllAgents(self, agent_type=None, rankId=None, uid_only=True):
        assert (rankId is not None) or (agent_type is not None)

        agentList = []

        if rankId is None:
            # If only agent_type is provided, return agents by type
            for agents in self.all_agents.get(agent_type, {}).values():
                agentList += agents

        elif agent_type is None:
            # If only rankID is provided, return agents by rank
            for agent_type in self.all_agents.keys():
                agentList += self.all_agents[agent_type].get(rankId, [])
        
        else:
            # If both rankID and agent_type are provided, return agents by rank and type
            agentList = self.all_agents[agent_type].get(rankId, [])
        
        if uid_only and agent_type != []:
            agentList = [agents[0] for agents in agentList]
            
        return agentList

    def getRealAgents(self, agent_type=None, rankId=None, uid_only=True):
        assert (rankId is not None) or (agent_type is not None)

        agentList = []

        if rankId is None:
            # If only agent_type is provided, return agents by type
            for agents in self.real_agents.get(agent_type, {}).values():
                agentList += agents

        elif agent_type is None:
            # If only rankID is provided, return agents by rank
            for agent_type in self.real_agents.keys():
                agentList += self.real_agents[agent_type].get(rankId, [])

        else:
            # If both rankID and agent_type are provided, return agents by rank and type
            agentList = self.real_agents[agent_type].get(rankId, [])
        
        if uid_only and agent_type != []:
            agentList = [agents[0] for agents in agentList]
            
        return agentList

    def getRandomRealAgent(self, agent_type=None, rankId=None, uid_only=True):
        assert (rankId is not None) or (agent_type is not None)
        
        agentList = self.getRealAgents(agent_type, rankId, uid_only)
        
        return random.choice(agentList)
    
    def getRandomAllAgent(self, agent_type=None, rankId=None, uid_only=True):
        assert (rankId is not None) or (agent_type is not None)
        
        agentList = self.getAllAgents(agent_type, rankId, uid_only)
    
        return random.choice(agentList)

def paramLoader(paramDict, randomGenerator=None, isGhost=False):
    newDict = {}

    if paramDict is None:
        return newDict
    else:
        if randomGenerator is not None:
            randomGenerator = randomGenerator
        else:
            randomGenerator = np.random.default_rng()
            
        for key, value in paramDict.items():
            if isinstance(value, list):
                min_value, max_value, dataType = value
                if isGhost:
                    newDict[key] = min_value
                else:
                    if dataType == 'int':
                        new_value = randomGenerator.integers(min_value, max_value, endpoint=True)
                    elif dataType == 'float':
                        new_value = min_value + randomGenerator.random() * (max_value - min_value)
                    else:
                        raise ValueError('When defining random initial values in a yaml file, we only support int or float')
                    
                    newDict[key] = new_value
            else:
                newDict[key] = value
    return newDict
                
                
                

In [ ]:
from collections import deque


class Model:
    
    def __init__(self, comm, params: Dict):
        self.params = params
        self.rank    = comm.Get_rank()
        self.rankNum = comm.Get_size()
        self.rng = rng

        self.runner = schedule.init_schedule_runner(comm)
        self.context = ctx.SharedContext(comm)


        self.totalProduction=[]
        self.totalCostOfProduction=[]
        self.totalCostOfUnusedFactors=[]
        self.totalInvGoodsRevenues=[]
        self.totalConsGoodsRevenues=[]
        self.totalInvGoodsInventories=[]
        self.totalConsGoodsInventories=[]
        self.totalInProgressInvGoodsInventories=[]
        self.totalInProgressConsGoodsInventories=[]
        self.totalLostProduction=[]
        self.totalCostOfLostProduction=[]
        self.updatedLabor=[]
        self.updatedCapital=[]
        self.totalGrossInvestmentQ=[]
        self.firmData={}
        
        self.theCentralPlannerReporter=0
        self.theCentralPlannerReporterGhostList=[]
        
        self.theCentralPlanner=0
        self.theCentralBank=0
        
        self.theCreditMarket = None
        self.theConsumptionMarket = None
        self.theLaborMarket = None

        self.bankData={}
        self.bankStockData = {}
        self.agent_cache = {}
        self.restore_agent_template = {}

        
        self.global_agent_uid = GlobalContainer(params)

        #the context and the runner are created in step 1 
      
        self.runner.schedule_event(          0.0,     self.initGhosts) 
        self.runner.schedule_event(          0.0,     self.initInvestmentGoodPrices) 
        
        self.runner.schedule_repeating_event(0.0,  1, self.counter)
        

        self.runner.schedule_repeating_event(0.03, 1, self.bankPayLoan)
        self.runner.schedule_repeating_event(0.04, 1, self.bankConcludingInformation)
        self.runner.schedule_repeating_event(0.05, 1, self.bankGetCredit)
        self.runner.schedule_repeating_event(0.06, 1, self.bankConcludingInformation)
        
        self.runner.schedule_repeating_event(0.20, 1, self.laborMarket)
        self.runner.schedule_repeating_event(0.21, 1, self.plannerPreparingActions)
        self.runner.schedule_repeating_event(0.22, 1, self.plannerDiffusingProductionOrders)
        self.runner.schedule_repeating_event(0.23, 1, self.firmsProducing)
        self.runner.schedule_repeating_event(0.24, 1, self.plannerPreparingAndMakingDistributionOfInvGoods)
        self.runner.schedule_repeating_event(0.25,  1, self.firmsConcludingProduction)
        # self.runner.schedule_repeating_event(0.26, 1, self.firmsMakingFinancialTransactionsRelatedToCosts)
        
        self.runner.schedule_repeating_event(0.4,  1, self.plannerGeneratingDemandOrders) #invGoods for next period investments
        
        # apply comsumptionMarket instead of the centralPlanner
        self.runner.schedule_repeating_event(0.41,  1, self.comsumptionMarket) #invGoods for next period investments
        
        # use centralPlanner to mimic the comsumptionMarket
        # self.runner.schedule_repeating_event(0.42, 1, self.firmsMakingFinancialTransactionsRelatedToRevenues)
        
        self.runner.schedule_repeating_event(0.50, 1, self.bankConcludingInformation)
        self.runner.schedule_repeating_event(0.51, 1, self.creditMarket)
        
        self.runner.schedule_repeating_event(0.7, 1, self.bankConcludingInformation)
        self.runner.schedule_repeating_event(0.71,  1, self.enterprisesMakingBalancesheet) #enterprises=firms+banks
        
        self.runner.schedule_stop(params['howManyCycles'])
        self.runner.schedule_end_event(self.finish)
        
        #-------------------------- ini agents -----------------------------------
        
        self.iniAgents()
        
        householdUIds = self.global_agent_uid.getRealAgents(params['HOUSEHOLD_TYPE'], self.rank)
        householdCount = 0

        #-------------------------- allocate resource to agents-----------------------------------
        
        # Initialize central banks
        
        self.theCentralBank = self.get_unique_agent(agent_type=params['CEN_BANK_TYPE'])
        
        # Initialize household
        for uid, (paramGroup, resourceGroup, isGlobal) in self.global_agent_uid.getRealAgents(params['HOUSEHOLD_TYPE'], self.rank, uid_only=False):
            ahousehold = self.context.agent(uid)
            resourceConfig = params['agent_resources'][uid[1]][resourceGroup]
            ini_deposit = resourceConfig['dp_min']
            # ini_deposit = resourceConfig['dp_min'] + rng.random()*(resourceConfig['dp_max'] -resourceConfig['dp_min'])
            # get a random bank to allocate resource
            bank_uid  = self.global_agent_uid.getRandomAllAgent(agent_type=params['COMM_BANK_TYPE'], rankId= self.rank)
            if bank_uid[2] == self.rank:
                aBank = self.context.agent(bank_uid)
            else:
                aBank = self.context.ghost_agent(bank_uid)
            
            
            deposit = Deposit(ini_deposit, ahousehold, aBank)
            ahousehold.localStocks.DEPOSIT.append(deposit)
            aBank.localStocks.DEPOSIT.append(deposit)
            
            ahousehold.lagValues.EMPLOYED = deque(np.zeros(4),maxlen=4)
            ahousehold.wage = 1
    
        # allocate reserves for real and ghost banks
        for uid, (paramGroup, resourceGroup, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=params['COMM_BANK_TYPE'], uid_only=False):
            resourceConfig = params['agent_resources'][uid[1]][resourceGroup]
            # ini_deposit = resourceConfig['rs_min'] + rng.random()*(resourceConfig['rs_max'] -resourceConfig['rs_min'])
            ini_deposit = resourceConfig['rs_min']

            if uid[2] == self.rank:
                aBank = self.context.agent(uid)
                reserve = Reserve(ini_deposit, aBank, self.theCentralBank)
                aBank.localStocks.RESERVE.append(reserve)
                self.theCentralBank.localStocks.RESERVE.append(reserve)
            elif uid[2] != self.rank and isGlobal:
                aBank = self.context.ghost_agent(uid)
                # ini with 0
                reserve = Reserve(0, aBank, self.theCentralBank)
                aBank.localStocks.RESERVE.append(reserve)
                self.theCentralBank.localStocks.RESERVE.append(reserve)
                
        
        # Initialize deposit for real and ghost CentralPlanner
        
        self.theCentralPlanner, (cp_paramGroup, cp_resourceGroup, cp_isGlobal) = self.get_unique_agent(agent_type=params['CEN_PLANNER_TYPE'], agent_only=False)
        cp_resourceConfig = params['agent_resources'][self.theCentralPlanner.uid[1]][resourceGroup]
        # select first global bank as the bank for centralPlanner (in the future, the centralPlanner will delete in decentralize model)
        ini_deposit = cp_resourceConfig['dp']

        fist_global_bank = None
        for uid, (paramGroup, resourceGroup, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=params['COMM_BANK_TYPE'], uid_only=False):
            if isGlobal:
                fist_global_bank = uid

        if fist_global_bank[2] == self.rank:
            aBank = self.context.agent(fist_global_bank)
            deposit = Deposit(ini_deposit, self.theCentralPlanner, aBank)
            self.theCentralPlanner.localStocks.DEPOSIT.append(deposit)
            aBank.localStocks.DEPOSIT.append(deposit)
        else:
            aBank = self.context.ghost_agent(fist_global_bank)
            deposit = Deposit(0, self.theCentralPlanner, aBank)
            self.theCentralPlanner.localStocks.DEPOSIT.append(deposit)
            aBank.localStocks.DEPOSIT.append(deposit)
            
        # Initialize reserves for real and ghost Firm
        for aFirm in self.context.agents(params['FIRM_TYPE']):
            # get a random bank to allocate resource
            bank_uid  = self.global_agent_uid.getRandomAllAgent(agent_type=params['COMM_BANK_TYPE'], rankId= self.rank)

            if bank_uid[2] == self.rank:
                aBank = self.context.agent(bank_uid)
            else:
                aBank = self.context.ghost_agent(bank_uid)

            deposit = Deposit(aFirm.capital, aFirm, aBank)
            aFirm.localStocks.DEPOSIT.append(deposit)
            aBank.localStocks.DEPOSIT.append(deposit)
            
            aFirm.localStocks.CONS_GOOD.append(ConsumptionGood(0, 1, 0, 100, 1, aFirm, aFirm))
            for _ in range(aFirm.laborExpect):
                householdUId = householdUIds[householdCount]
                ahousehold = self.context.agent(householdUId)
                aFirm.employees.append(ahousehold)
                ahousehold.employer = aFirm
                ahousehold.lagValues.EMPLOYED.appendleft(1)
                householdCount+=1
            
            aFirm.theCentralPlanner = self.theCentralPlanner
            
        # print(householdCount)
        
        
        # Initialize Markets
        self.theCreditMarket = self.get_unique_agent(agent_type=params['CREDIT_MARKET_TYPE'])
        self.theConsumptionMarket = self.get_unique_agent(agent_type=params['CONSUMPTION_MARKET_TYPE'])
        self.theLaborMarket = self.get_unique_agent(agent_type=params['LABOR_MARKET_TYPE'])

    
    def t(self):
        return int(self.runner.schedule.tick)
    
    # get unique agent in current rank (real or ghost)
    def get_unique_agent(self, agent_type, agent_only=True):
        uid, (paramGroup, resourceGroup, isGlobal) = self.global_agent_uid.getAllAgents(agent_type=agent_type, rankId=self.rank, uid_only=False)[0]

        if uid[2] == self.rank:
            agent = self.context.agent(uid)
        else:
            agent = self.context.ghost_agent(uid)
        
        if agent_only:
            return agent
        else:
            return agent, (paramGroup, resourceGroup, isGlobal)
    
    def restore_agent(self, agent_data: Tuple):

        uid=agent_data[0]
        
        # adaptive old version
        if uid[1] in self.restore_agent_template:
            isGlobal, paramGroup = agent_data[1][0], agent_data[1][1]
            agent_param = paramLoader(params['agent_params'][uid[1]][paramGroup], isGhost=True)
     
        if uid[1] == params['FIRM_TYPE']:
        
            if uid in self.agent_cache: 
                tmp = self.agent_cache[uid] # found
                tmp.labor = agent_data[1][0] #restore data
                tmp.capital = agent_data[1][1]
                tmp.minOrderDuration = agent_data[1][2]
                tmp.maxOrderDuration = agent_data[1][3]
                tmp.recipe = agent_data[1][4]
                tmp.laborProductivity = agent_data[1][5]
                tmp.maxOrderProduction = agent_data[1][6]
                tmp.assetsUsefulLife = agent_data[1][7]
                tmp.plannedMarkup = agent_data[1][8]
                tmp.orderObservationFrequency = agent_data[1][9]
                tmp.productionType = agent_data[1][10]
                tmp.sectorialClass = agent_data[1][11]
                
            else: #creation of an instance of the class with its data
                tmp = Firm(uid[0], uid[2],agent_data[1][0],agent_data[1][1],agent_data[1][2],agent_data[1][3],\
                           agent_data[1][4],agent_data[1][5],agent_data[1][6],agent_data[1][7],agent_data[1][8],\
                           agent_data[1][9],agent_data[1][10],agent_data[1][11])
                self.agent_cache[uid] = tmp
                
            return tmp
        
        
        if uid[1] in self.restore_agent_template:
            agentClass, _ = self.restore_agent_template[uid[1]]
            if uid in self.agent_cache: 
                raise ValueError('should not happen')
                tmp = agent_cache[uid] # found
    
            else: #creation of an instance of the class with its data
                tmp = agentClass(uid, self, isGlobal, paramGroup, **agent_param)
                self.agent_cache[uid] = tmp
    
            return tmp
        elif (uid[1] - 1000) in self.restore_agent_template:
            _, proxyAgent = self.restore_agent_template[uid[1]-1000]
            if uid in self.agent_cache: 
                tmp = self.agent_cache[uid] # found
    
            else: #creation of an instance of the class with its data
                tmp = proxyAgent(uid[0], uid[1], uid[2])
                self.agent_cache[uid] = tmp
            return tmp
        else:
            raise ValueError(f'please ini agnet type {uid[1]} in initAgents')
        
    
    def initGhosts(self):
        pass
    
    def iniGhostAgent(self, agentType, reporterClass):

        # call bank ghost if it is a global agent
        ghostToRequest = []

        for agentUid, (paramGroup, resourceGroup, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=agentType, uid_only=False):
            if agentUid[2] != self.rank and isGlobal:
                ghostToRequest.append((agentUid, agentUid[2]))
                
        self.context.request_agents(ghostToRequest,self.restore_agent)

        
        # call reporter
        for agentUid, (paramGroup, resourceGroup, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=agentType, uid_only=False):
            if agentUid[2] != self.rank and isGlobal:
                agent = self.context.ghost_agent(agentUid)
                
                uniqe_id = (agentUid[0] + agentUid[2]) * (agentUid[0] + agentUid[2] + 1) // 2 + agentUid[2]
                angentReporter = reporterClass(uniqe_id, agentType+1000, self.rank) #local_id=0, rank=rank
                angentReporter.connectAgent = agent
                self.context.add(angentReporter)

        # call reporter ghost
        reporterToRequest = []

        for agentUid, (paramGroup, resourceGroup, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=agentType, uid_only=False):
            if agentUid[2] == self.rank and isGlobal:
                for i in range(rankNum):
                    if i != self.rank:
                        uniqe_id = (agentUid[0] + agentUid[2]) * (agentUid[0] + agentUid[2] + 1) // 2 + agentUid[2]
                        reporterToRequest.append(((uniqe_id, agentType+1000, i), i))

        self.context.request_agents(reporterToRequest,self.restore_agent)

        # connect the main agent with reporter ghost
        for agentUid, (paramGroup, resourceGroup, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=agentType, uid_only=False):
            if agentUid[2] == self.rank and isGlobal:
                agent = self.context.agent(agentUid)
                for i in range(rankNum):
                    if i != self.rank:
                        uniqe_id = (agentUid[0] + agentUid[2]) * (agentUid[0] + agentUid[2] + 1) // 2 + agentUid[2]
                        agent.reporterGhostList.append(self.context.ghost_agent((uniqe_id, agentType+1000, i)))

    def iniAgents(self):
        
        self.iniAgent(params['FIRM_TYPE'], Firm, AgentReporter)
        self.iniAgent(params['CEN_PLANNER_TYPE'], CentralPlanner, AgentReporter)
        self.iniAgent(params['COMM_BANK_TYPE'], CommercialBank, AgentReporter)
        self.iniAgent(params['CEN_BANK_TYPE'], CentralBank, AgentReporter)
        self.iniAgent(params['HOUSEHOLD_TYPE'], Household, AgentReporter)
        
        self.iniAgent(params['CREDIT_MARKET_TYPE'], CreditMarket, MarketReporter)
        self.iniAgent(params['CONSUMPTION_MARKET_TYPE'], ComsumptionMarket, MarketReporter)
        self.iniAgent(params['LABOR_MARKET_TYPE'], LaborMarket, MarketReporter)

    
    def iniAgent(self, agentType, agentClass, proxyClass):
        
        # initial agent entity
        for agentUid, (paramGroup, resourceGroup, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=agentType, rankId=self.rank, uid_only=False):
            agent_param = paramLoader(params['agent_params'][agentUid[1]][paramGroup], randomGenerator=rng)
            agent = agentClass(agentUid, model=self, isGlobal=isGlobal, paramGroup=paramGroup,  **agent_param)
            self.context.add(agent)
        
        # add restore agent
        self.restore_agent_template[agentType] = (agentClass, proxyClass)
        
        # initial ghost agent if needed
        self.iniGhostAgent(agentType, proxyClass)
        
    
    #initialize investment good prices
    def initInvestmentGoodPrices(self):
        self.investmentGoodPrices=[0]*len(params['investmentGoods'])
        
        for anInvGoodType in range(len(params['investmentGoods'])):
            count=0
            for aFirm in self.context.agents(agent_type=params['FIRM_TYPE']):
                if aFirm.productionType == params['investmentGoods'][anInvGoodType]:
                    self.investmentGoodPrices[anInvGoodType]+=aFirm.estimatingInitialPricePerProdUnit()
                    count+=1
            if count != 0: self.investmentGoodPrices[anInvGoodType]/=count
        
        if not any(self.investmentGoodPrices): 
            print("\nThere are no investment goods!")
            sys.exit(0)
        
        for aFirm in self.context.agents(agent_type=params['FIRM_TYPE']):
            aFirm.settingCapitalQ(self.investmentGoodPrices)
            if aFirm.uid[0]==0: print("rank",self.rank,"Initial price of durable productive goods per unit",\
                                      aFirm.priceOfDurableProductiveGoodsPerUnit, flush=True) #as an info to the user
                

    #count the cycles number
    def counter(self):
        if int(self.t()) % params["tickNumber.betweenChecks"] == 0 and self.t()>9: 
            print("rank", self.rank, "tick", self.t(), \
                  "proportionalValue",self.theCentralPlanner.proportionalValue, flush=True)
            

    def plannerPreparingActions(self): 
        #workingMultiRank, rules are the same
        self.theCentralPlanner.preparingActions(self) # self here is the model instance
        # step made in paraller independly in all the ranks using the infos of plannerMethods.py
        # 
        ##
        ##sinchronize ghosts
        ##
         #theCentralPlanner diffuse infos to its ghosts
                                                           #from rank 0 to the other ranks (currentry nothing
                                                           #interesting)
                                                           #theCentralPlannerReporter send infos to its ghost
                                                           # from rank !=0 to rank 0
        if self.rankNum > 1 and self.context.contains_type(agent_type=params['CEN_PLANNER_TYPE'] + 1000):
            for aReporter in self.context.agents(agent_type=params['CEN_PLANNER_TYPE'] + 1000):
                aReporter.reciveInformationLastCol()

        
        if self.rankNum > 1: self.context.synchronize(self.restore_agent)
        
        #test
        #if rank==0:
        #    for i in range(1,rankNum):
        #        print(rank, t(), self.theCentralPlannerReporterGhostList[i-1].informationTableLastCol,flush=True)

        #add data collected from central planner reporter of ranks > 0 to the central planner of rank 0 data
        
        # 
        self.theCentralPlanner.mergeInformationTableData()
        
        
        # if self.rank==0:
        #     print('old', self.theCentralPlanner.informationTableMultirank)
        #     print('new', self.theCentralPlanner.globalFlows[:4])
        
    def plannerDiffusingProductionOrders(self):
        ###
        ###parallel independent operations if multirank
        ###
        self.theCentralPlanner.diffusingProductionOrders(self)
    
    def firmsProducing(self):
        rowNumber = len(params['ini_agents_uniform'][params['FIRM_TYPE']]['local_distribute'])
        # print(rowNumber)
        self.totalProduction.append([0]*rowNumber) #for each cycle adds a sub-list of lenght number of firm class types
        self.totalCostOfProduction.append([0]*rowNumber)
        self.totalCostOfUnusedFactors.append([0]*rowNumber)
        self.totalInvGoodsInventories.append([0]*rowNumber)
        self.totalInProgressInvGoodsInventories.append([0]*rowNumber)
        self.totalConsGoodsInventories.append([0]*rowNumber)
        self.totalInProgressConsGoodsInventories.append([0]*rowNumber)
        self.totalLostProduction.append([0]*rowNumber)
        self.totalCostOfLostProduction.append([0]*rowNumber)
        self.updatedLabor.append([0]*rowNumber)
        self.updatedCapital.append([0]*rowNumber)
        self.totalGrossInvestmentQ.append([0]*rowNumber)
        
        for aFirm in self.context.agents(agent_type=params['FIRM_TYPE']): #SHUFFLE to make them acting in random order
            aFirm.produce(self.t(), rng)
                       
            
    def plannerPreparingAndMakingDistributionOfInvGoods(self):

        self.theCentralPlanner.askFirmsInvGoodsDemand(self)
        ###
        ###sinchronize ghosts
        if self.rankNum > 1 and self.context.contains_type(agent_type=params['CEN_PLANNER_TYPE'] + 1000):
            for aReporter in self.context.agents(agent_type=params['CEN_PLANNER_TYPE'] + 1000):
                aReporter.reciveInformationLastCol()
        
        ###
        if rankNum > 1: self.context.synchronize(self.restore_agent)

        #test
        #if rank==0:
        #    for i in range(1,rankNum):
        #        print("from M, rank=",rank,"t=", t(), \
        #              self.theCentralPlannerReporterGhostList[i-1].invGoodsDemandList,flush=True)

        ###
        ###MULTIRANK
        ###
        #add data collected from central planner reporter of ranks > 0 to the central planner of rank 0 data
        
        self.theCentralPlanner.mergeInformationTableData()
        
        # if rank == 0:
        #     print('old1', (self.theCentralPlanner.allFirmsDesiredCapitalQsubstitutionsMultirank,
        #         self.theCentralPlanner.allFirmsRequiredCapitalQincrementMultirank,
        #         self.theCentralPlanner.allFirmsDesiredCapitalSubstitutionsMultirank,
        #         self.theCentralPlanner.allFirmsRequiredCapitalIncrementMultirank))
        #     print('new1', self.theCentralPlanner.globalFlows[4:])
            

        #determining and diffusing (if multirank) the proportionalValue to be used in the case "propotionally"
        if self.rank==0: self.theCentralPlanner.setProportionalValue(self)
        ###
        ###sinchronize ghosts
        ###
        if rankNum > 1: self.context.synchronize(self.restore_agent)
        
            
        self.theCentralPlanner.executeInvestmentGoodsDemandFromFirms(self)
        
    
    
    def firmsConcludingProduction(self):
        for aFirm in self.context.agents(params['FIRM_TYPE']):
            
            tupleOfProductionResults = aFirm.concludeProduction()

            self.totalProduction[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[0]
            self.totalCostOfProduction[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[1]
            self.totalCostOfUnusedFactors[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[2]
            
            if not aFirm.productionType in params["investmentGoods"]: 
                self.totalConsGoodsInventories[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[3]
                self.totalInProgressConsGoodsInventories[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[4]  
            else: 
                self.totalInvGoodsInventories[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[3]
                self.totalInProgressInvGoodsInventories[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[4]
            
            #here we will need to separate invGoods and consGoods inventories (and in progr inventories)
            #same for revenues, to be added here to the series
            self.totalLostProduction[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[5]
            self.totalCostOfLostProduction[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[6]
            self.updatedLabor[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[7]
            self.updatedCapital[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[8]
            self.totalGrossInvestmentQ[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[9]

    def firmsMakingFinancialTransactionsRelatedToCosts(self):
        for aBank in self.global_agent_uid.getAllAgents(agent_type=params['COMM_BANK_TYPE'], rankId=self.rank):
            if aBank[2] == self.rank:
                aBank = self.context.agent(aBank)
            else:
                aBank = self.context.ghost_agent(aBank)
            aBank.payDepositInterests()

        for aFirm in self.context.agents(params['FIRM_TYPE']):
            aFirm.payInterests(self.t())
            # aFirm.payWage()
    
    def plannerGeneratingDemandOrders(self):
        #currently independent processes, so also operating multirank
        self.theCentralPlanner.generateDemandOrders(self)
        
    def firmsMakingFinancialTransactionsRelatedToRevenues(self):
        
        for aFirm in self.context.agents(agent_type=params['FIRM_TYPE']):

            targetDeposit = aFirm.localStocks.DEPOSIT[0]
            sourceDeposit = aFirm.theCentralPlanner.Deposits[0]

            sourceDeposit.liabilityHolder.transfer(sourceDeposit, targetDeposit, aFirm.revenues)
            
        
        
    def bankConcludingInformation(self):
        if rankNum > 1: 
            for aBankReporter in self.context.agents(agent_type=params['COMM_BANK_TYPE'] + 1000):
                aBankReporter.reciveInformationLastCol()
            
            self.context.synchronize(self.restore_agent)

        for aBank in self.context.agents(agent_type=params['COMM_BANK_TYPE']):
            aBank.mergeInformationTableData()
        
        if rankNum > 1: 
            self.context.synchronize(self.restore_agent)

    def bankGetCredit(self):
        for aBank in self.context.agents(agent_type=params['COMM_BANK_TYPE']):
            advancesDemand = aBank.getAdvanceDemand()

            advances = Loan(value=advancesDemand, interestRate=self.theCentralBank.advanceInterestRate, startTick=self.t(), length=1,
             ObservePeriod=params['loanObservationFrequency'], assetHolder=self.theCentralBank, liabilityHolder=aBank)



            aBank.localStocks.ADVANCE.append(advances)
            self.theCentralBank.localStocks.ADVANCE.append(advances)

            if len(aBank.localStocks.RESERVE) == 1:
                aBank.localStocks.RESERVE[0].value += advancesDemand
            else:
                print('not happen')
                depositCB = Deposit(advancesDemand, aBank, self.theCentralBank)
                aBank.localStocks.RESERVE.append(depositCB)
                self.theCentralBank.localStocks.RESERVE.append(depositCB)

    def bankPayLoan(self):
        for aBank in self.context.agents(agent_type=params['COMM_BANK_TYPE']):
            aBank.payInterests(self.t())
    

    def enterprisesMakingBalancesheet(self):
        rowNumber = len(params['ini_agents_uniform'][params['FIRM_TYPE']]['local_distribute'])
        
        self.totalInvGoodsRevenues.append([0]*rowNumber)
        self.totalConsGoodsRevenues.append([0]*rowNumber)
        
        for aFirm in self.context.agents(agent_type=params['FIRM_TYPE']):
            aFirm.makeBalancesheet(self.t())
            self.totalConsGoodsRevenues[self.t()][aFirm.sectorialClass] += aFirm.myBalancesheet[self.t(), 3]
            self.totalInvGoodsRevenues[self.t()][aFirm.sectorialClass] += aFirm.myBalancesheet[self.t(), 4]
        
        for aBank in self.context.agents(agent_type=params['COMM_BANK_TYPE']):
            aBank.makeBalancesheet(self.t())
        
        
        for agent_uid in self.global_agent_uid.getAllAgents(rankId=self.rank):
            if agent_uid[1] >= 100:
                continue
            if agent_uid[2] == self.rank:
                agent = self.context.agent(agent_uid)
            else:
                agent = self.context.ghost_agent(agent_uid)

            agent.resetFlows()
        
        
        for aFirm in self.context.agents(agent_type=params["FIRM_TYPE"]):
            if not np.isclose(aFirm.localStocks.CONS_GOOD[0].quantity, aFirm.inventories, rtol=1e-5):
                print(aFirm.localStocks.CONS_GOOD[0].quantity, aFirm.inventories)
        
        
        for aFirm in self.context.agents(agent_type=params['FIRM_TYPE']):
            if not np.isclose(aFirm.localStocks.DEPOSIT[0].value, aFirm.capital + aFirm.capitalQ, rtol=1e-5):
                print(aFirm.localStocks.DEPOSIT[0].value, aFirm.capital + aFirm.capitalQ)

    
    def comsumptionMarket(self):
        demanderList = self.context.agents(agent_type=params['HOUSEHOLD_TYPE'])
        supplierList = self.context.agents(agent_type=params['FIRM_TYPE'])
        demanderUids = []
        for demander in demanderList:
            demanderUids.append(demander.uid)
            asset = demander.localStocks.DEPOSIT[0].value
            
            consumption_demand = demander.computeConsumptionDemand()
            # demand goods, asset
            self.theConsumptionMarket.collectDemandInfo(demander.uid, [consumption_demand, asset])
            
        
        supplierUids = []
        for supplier in supplierList:
            supplierUids.append(supplier.uid)
            
            # inventory = supplier.ConsumptionGoods[0]
            
            # self.theCreditMarket.collectSupplyInfo(supplier.uid, [params['centralPlannerPriceCoefficient'], inventory.quantity])
            
            self.theConsumptionMarket.collectSupplyInfo(supplier.uid, [params['centralPlannerPriceCoefficient'], supplier.inventory_delta])
        
        self.theConsumptionMarket.collectDemander(demanderUids)
        self.theConsumptionMarket.collectSupplyer(supplierUids)

        
        if self.theConsumptionMarket.isGlobal and self.theConsumptionMarket.uid[2] != self.rank: 
            for aMarketReporter in self.context.agents(agent_type=params['CONSUMPTION_MARKET_TYPE'] + 1000):
                aMarketReporter.reciveInformationLastCol()

        if self.theConsumptionMarket.isGlobal: self.context.synchronize(self.restore_agent)
        
        if self.theConsumptionMarket.uid[2] == self.rank:
            self.theConsumptionMarket.mergeInformationTableData()
            self.theConsumptionMarket.execute()
        
        if self.theConsumptionMarket.isGlobal: self.context.synchronize(self.restore_agent)
        
        matchResult = self.theConsumptionMarket.getMatchResult(self.rank)
        if matchResult != {}:
            
            for demanderUid, supplierInfos in matchResult.items():
                if demanderUid[2] == self.rank:
                    demander = self.context.agent(demanderUid)
                else:
                    raise ValueError('mismatch in consumption market: wrong rank')

                    
                for supplierUid, matchInfo in supplierInfos:
                    if supplierUid[2] == self.rank:
                        supplier = self.context.agent(supplierUid)
                    else:
                        raise ValueError('mismatch in consumption market: wrong rank: consumption market only work locally')

                    
                    price, quantity = matchInfo
                    
                    # only apply the deposit transform
                    
                    inventory = supplier.localStocks.CONS_GOOD[0]

                    good = ConsumptionGood(price, quantity, self.t(), 4, params['loanObservationFrequency'], demander, supplier)
                    # 
                    demander.localStocks.CONS_GOOD.append(good)
                    inventory.quantity -= quantity
                    # 
                    # demanderDeposits = demander.localStocks.DEPOSIT[0]
                    # supplierDeposits = supplier.localStocks.DEPOSIT[0]
                    # 
                    # demanderDeposits.liabilityHolder.transfer(demanderDeposits, supplierDeposits, price * quantity)
        
    
    def creditMarket(self):
        
        demanderList = self.context.agents(agent_type=params['FIRM_TYPE'])
        supplierUids = self.global_agent_uid.getAllAgents(agent_type=params['COMM_BANK_TYPE'], rankId=self.rank)
        demanderUids = []
        for demander in demanderList:
            demanderUids.append(demander.uid)
            
            capitalQChange = demander.investmentGoodsGivenByThePlanner[0] + demander.investmentGoodsGivenByThePlanner[1]
            capitalChange = demander.investmentGoodsGivenByThePlanner[2] + demander.investmentGoodsGivenByThePlanner[3]
            self.theCreditMarket.collectDemandInfo(demander.uid, [capitalQChange + capitalChange])
        
        for supplier in self.context.agents(agent_type=params['COMM_BANK_TYPE']):
            supplier.getCreditSupply()
            self.theCreditMarket.collectSupplyInfo(supplier.uid, [supplier.loanSupply])
        
        self.theCreditMarket.collectDemander(demanderUids)
        self.theCreditMarket.collectSupplyer(supplierUids)

        if self.theCreditMarket.isGlobal and self.theCreditMarket.uid[2] != self.rank:
            for aMarketReporter in self.context.agents(agent_type=params['CREDIT_MARKET_TYPE'] + 1000):
                aMarketReporter.reciveInformationLastCol()
            
        if self.theCreditMarket.isGlobal: self.context.synchronize(self.restore_agent)
        
        if self.theCreditMarket.uid[2] == self.rank:
            self.theCreditMarket.mergeInformationTableData()
            self.theCreditMarket.execute()
        
        if self.theCreditMarket.isGlobal: self.context.synchronize(self.restore_agent)
        
        matchResult = self.theCreditMarket.getMatchResult(self.rank)
        if matchResult != {}:
            for demanderUid, supplierInfos in matchResult.items():
                if demanderUid[2] == self.rank:
                    demander = self.context.agent(demanderUid)
                else:
                    raise ValueError('mismatch in credit market: wrong rank')

                    
                for supplierUid, (matchInfo) in supplierInfos:
                    if supplierUid[2] == self.rank:
                        supplier = self.context.agent(supplierUid)
                    else:
                        supplier = self.context.ghost_agent(supplierUid)

                    value = matchInfo
                    loan = Loan(value, supplier.loanInterestRate, self.t(), 4, params['loanObservationFrequency'], supplier, demander)
                    
                    demander.localStocks.LOAN.append(loan)
                    supplier.localStocks.LOAN.append(loan)
                    
                    demanderDeposits = demander.localStocks.DEPOSIT[0]
                    if demanderDeposits.liabilityHolder == supplier:
                        demanderDeposits.value += value
                    else:
                        demanderDeposits.value += value
                        supplier.localStocks.RESERVE[0].value -= value
                        demanderDeposits.liabilityHolder.localStocks.RESERVE[0].value += value
        
    
    def laborMarket(self):
        demanderList = self.context.agents(agent_type=params['HOUSEHOLD_TYPE'])
        supplierList = self.context.agents(agent_type=params['FIRM_TYPE'])
        demanderUids = []
        supplierUids = []
        
        for supplier in supplierList:
            laborDemand = supplier.computeLaborDemand()
            if laborDemand > 0:
                supplierUids.append(supplier.uid)
                self.theLaborMarket.collectSupplyInfo(supplier.uid, [laborDemand])
            
        for demander in demanderList:
            demander.computeWage()
            
            # demand goods, asset
            if demander.employer is None:
                demanderUids.append(demander.uid)
                self.theLaborMarket.collectDemandInfo(demander.uid, [1])
        
        self.theLaborMarket.collectDemander(demanderUids)
        self.theLaborMarket.collectSupplyer(supplierUids)

        if self.theLaborMarket.isGlobal and self.theLaborMarket.uid[2] != self.rank:
            for aMarketReporter in self.context.agents(agent_type=params['LABOR_MARKET_TYPE'] + 1000):
                aMarketReporter.reciveInformationLastCol()

        if self.theLaborMarket.isGlobal: self.context.synchronize(self.restore_agent)
        
        if self.theLaborMarket.uid[2] == self.rank:
            self.theLaborMarket.mergeInformationTableData()
            self.theLaborMarket.execute()
        
        if self.theLaborMarket.isGlobal: self.context.synchronize(self.restore_agent)
        
        matchResult = self.theLaborMarket.getMatchResult(self.rank)
        if matchResult != {}:
            for demanderUid, supplierInfos in matchResult.items():
                if demanderUid[2] == self.rank:
                    demander = self.context.agent(demanderUid)
                else:
                    raise ValueError('mismatch in labor market: wrong rank')

                    
                for supplierUid, matchInfo in supplierInfos:
                    if supplierUid[2] == self.rank:
                        supplier = self.context.agent(supplierUid)
                    else:
                        raise ValueError('mismatch in labor market: wrong rank: labor market only work locally')
                    
                    supplier.employees.append(demander)
                    demander.employer = supplier
        

        
                
        
    #finish
    def finish(self):
        
        print("cpu time - calculating phase", Tc(), "rank", self.rank, flush=True)

        print(self.context.size(agent_type_ids=[params['FIRM_TYPE']]))
        
        # infos for data_analysis*.ipynb
        with open('plotInfo.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow((params["log_file_root"],rankNum,\
                             self.context.size(agent_type_ids=[params['FIRM_TYPE']])[params['FIRM_TYPE']]))
        
        
        #series____________________________________________________
        
        names=["_total_production_","_total_cost_of_production_","_total_cost_of_unused_factors_",\
               "_total_inv_goods_revenues_", "_total_cons_goods_revenues_",\
               "_total_inv_goods_inventories_","_total_in_progress_inv_goods_inventories_",\
               "_total_cons_goods_inventories_","_total_in_progress_cons_goods_inventories_",\
               "_total_lost_production_","_total_cost_of_lost_production_","_updatedLabor_","_updatedCapital_",\
               "_total_grossInvestmentQ_"]
        contents=[self.totalProduction,self.totalCostOfProduction,self.totalCostOfUnusedFactors,
                  self.totalInvGoodsRevenues, self.totalConsGoodsRevenues, 
                  self.totalInvGoodsInventories,self.totalInProgressInvGoodsInventories,
                  self.totalConsGoodsInventories,self.totalInProgressConsGoodsInventories,
                  self.totalLostProduction,self.totalCostOfLostProduction,
                  self.updatedLabor,self.updatedCapital, self.totalGrossInvestmentQ]
        
        for s in range(len(names)):
            with open(params["log_file_root"]+names[s]+str(self.rank)+'.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                for k in range(params["howManyCycles"]):
                    writer.writerow(contents[s][k])

        
        #balancesheets______________________________________________
        #via pickle
        
        #creating a dictionary of firm dataframes
        #firmData={} defined in __init__
        colNames=["firm class type", "initial inventories","total costs", "revenuesCons", "revenuesInv", "consGoods inventories",\
       "invGoods inventories",  "consGoods in progr. inventories", "invGoods in progr. inventories", "profits", \
          "added value", "total production", "cost of production", "cost of unused factors", "total lost production", \
          "total cost of lost production", "cost of labor", "cost of capital", "gross investment in Q",\
            "production type"]
        

        for aFirm in self.context.agents(agent_type=params["FIRM_TYPE"]):
            self.firmData[aFirm.uid]=pd.DataFrame(aFirm.myBalancesheet)
            self.firmData[aFirm.uid].columns=colNames

        pickle.dump(self.firmData, open(params["log_file_root"]+'_balancesheetDict.p', "wb"))

        bankColNames=["total deposite", "total reserve", "total advances", "total loans"
        , "total interest deposit", "total interest advance", 'netWealth']
        
        
        for aBank in self.context.agents(agent_type=params["COMM_BANK_TYPE"]):
            self.bankData[aBank.uid]=pd.DataFrame(aBank.myBalancesheet)
            self.bankData[aBank.uid].columns=bankColNames

        pickle.dump(self.bankData, open(params["log_file_root"]+'_bankBalancesheetDict.p', "wb"))

        #workingMultiRank
        np.savetxt("plannerInfo.csv", self.theCentralPlanner.informationTable, delimiter=",")
        print("cpu time - finishing phase", Tc(), "rank", self.rank, flush=True)
        print("THE END!", flush=True)
    
    def start(self):
        self.runner.execute()

In [ ]:
# mpirun -n 4 ipython model1_general/model.ipynb
model = Model(comm, params)
model.start()


In [ ]:
print(rank, model.context.size([params['FIRM_TYPE']]))
# print(rank, len(model.global_agent_uid.getAllAgents(params['FIRM_TYPE'], rank)))
# print(rank, len(model.global_agent_uid.getRealAgents(params['FIRM_TYPE'], rank)))